In [1]:
import pandas as pd
from scipy.io import loadmat
import altair as alt
import numpy as np
import math
from scipy.io import loadmat
from src.spinorama.load import graph_melt
from src.spinorama.graph import graph_freq, graph_params_default
from src.spinorama.analysis import compute_cea2034

IndentationError: expected an indented block (load.py, line 243)

In [2]:
def parse_graph_freq_princeton_mat(mat, suffix):
    """ Suffix can be either H or V """
    ir_name = 'IR_{:1s}'.format(suffix)
    fs_name = 'fs_{:1s}'.format(suffix)
    # compute Freq                                                                                                                   
    timestep = 1./mat[fs_name]
    # hummm                                                                                                                          
    freq = np.fft.fftfreq(2**14, d=timestep)
    # reduce spectrum to 0 to 24kHz                                                                                                  
    # lg = 2**14                                                                                                                     
    # 24k is lgs = int(lg/4)                                                                                                         
    # 20k is at 3414                                                                                                                 
    lgs = 3414
    xs = freq[0][0:lgs]
    #    
    df = pd.DataFrame({'Freq': xs})
    # loop over measurements (skipping the 5 increments)
    for i in range(0, 72, 1):
        # extract ir                                                                                                                 
        ir = mat[ir_name][i]
        # compute FFT                                                                                                                
        y = np.fft.fft(ir)
        ys = np.abs(y[0:lgs])
        # check for 0 (from manual: 0 means not measured)                                                                            
        if ys.max() == 0.0:
            continue
        # apply formula from paper to translate to dbFS                                                                              
        ys = 105.+np.log10(ys)*20.
        # interpolate to smooth response                                                                                             
        # s = InterpolatedUnivariateSpline(xs, ys)                                                                                   
        # pretty print label, per 5 deg increment, follow klippel labelling                                                          
        ilabel =i*5
        if ilabel > 180: 
            ilabel = ilabel-360
        label = '{:d}°'.format(ilabel)
        if ilabel == 0:
            label = 'On Axis'
        df[label] = ys
    # sort columns in increasing angle order 
    def a2v(angle):
        if angle == 'Freq':
            return -1000
        elif angle == 'On Axis':
            return 0
        else:
            return int(angle[:-1])

    df = df.reindex(columns=sorted(df.columns, key=lambda a: a2v(a)))
    # check empty case
    if 'On Axis' not in df.keys():
        return None
    # precision of measurement is ok above 500
    return df[df.Freq>=500]

In [3]:
h_mat = loadmat("datas/Princeton/Avantgarde Acoustic Solo/Avantgarde_H_IR.mat")
v_mat = loadmat("datas/Princeton/Avantgarde Acoustic Solo/Avantgarde_V_IR.mat")

In [4]:
df = parse_graph_freq_princeton_mat(v_mat, 'V')
df is None

True

In [5]:
df = parse_graph_freq_princeton_mat(h_mat, 'H')
df

,Freq,On Axis,5°,10°,15°,20°,25°,30°,35°,40°,...,135°,140°,145°,150°,155°,160°,165°,170°,175°,180°
86,503.906250,88.680184,88.586274,88.440273,88.207657,87.865293,87.462634,86.949847,86.426490,85.789337,...,73.713033,73.708300,74.128973,74.708505,75.472328,76.230214,76.916420,77.426370,77.720839,77.810972
87,509.765625,88.585381,88.525556,88.385474,88.143055,87.849602,87.461242,87.025180,86.489356,85.921926,...,74.226974,74.342174,75.290591,75.393255,76.152078,76.883158,77.586941,78.088677,78.420882,78.539069
88,515.625000,88.349471,88.324169,88.190018,87.980951,87.687271,87.335597,86.883418,86.375033,85.837705,...,74.812764,75.208535,75.655202,76.549490,77.407062,78.204996,78.829333,79.310478,79.603095,79.717507
89,521.484375,88.075364,88.060069,87.951529,87.748149,87.459938,87.077159,86.678265,86.216401,85.724790,...,74.625355,75.149224,75.650294,77.112166,78.111391,79.017641,79.726454,80.244198,80.572675,80.724068
90,527.343750,87.784042,87.783088,87.702594,87.509448,87.233121,86.892712,86.473601,86.042377,85.555875,...,74.134623,74.873841,76.206230,77.175983,78.315059,79.288561,80.091572,80.685630,81.052557,81.235271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3409,19974.609375,62.406832,65.984691,69.715122,68.403113,69.816349,66.124404,65.847962,61.881460,59.403137,...,6.367370,25.394240,29.016549,30.783813,28.935092,22.603902,24.963538,22.390453,30.269924,26.842572
3410,19980.468750,62.154144,65.796843,69.688042,68.371097,69.812432,66.130832,65.850865,61.848369,59.334028,...,10.183515,26.691112,28.046928,30.644087,30.762696,19.560846,20.327542,16.383008,28.656155,29.087443
3411,19986.328125,61.952354,65.586978,69.629826,68.352085,69.789064,66.126367,65.834709,61.821980,59.282388,...,18.729348,25.941434,18.406744,30.310605,29.495719,16.175255,18.507731,23.385398,28.270748,26.751219
3412,19992.187500,61.762524,65.437166,69.587617,68.314352,69.754521,66.128730,65.757783,61.844135,59.283170,...,14.397082,25.426991,26.526182,28.225562,27.486851,20.871560,23.799266,26.047034,29.005649,25.152113
